In [ ]:
# Cell 1: Imports and Setup
import os
import numpy as np
import pandas as pd
import json
from tqdm import tqdm
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from sklearn.metrics import top_k_accuracy_score

print("✅ Imports loaded successfully!")
print(f"📊 TensorFlow version: {tf.__version__}")
print(f"🎮 GPU available: {len(tf.config.list_physical_devices('GPU')) > 0}")

In [ ]:
# Cell 2: Configuration (64 frames for complete signs)
TARGET_FRAMES = 64
N_FACE = 468
N_POSE = 33
N_LEFT_HAND = 21
N_RIGHT_HAND = 21
TOTAL_LANDMARKS = N_FACE + N_POSE + N_LEFT_HAND + N_RIGHT_HAND  # 543
INPUT_SHAPE = (TARGET_FRAMES, TOTAL_LANDMARKS * 3)  # (64, 1629)

# Dataset Paths (ISLR ONLY)
ISLR_TRAIN_PATH = "/kaggle/input/wlasl2000-landmarks/train.tfrecord"  # 23.36 GB
ISLR_VAL_PATH = "/kaggle/input/wlasl2000-landmarks/val.tfrecord"      # 7.96 GB
ISLR_MAPPING_PATH = "/kaggle/input/islr-mappings/sign_to_prediction_index_map.json"  # 250 words

print(f"✅ Configuration:")
print(f"   Target frames: {TARGET_FRAMES} ({TARGET_FRAMES/30:.1f}s @ 30fps)")
print(f"   Total landmarks: {TOTAL_LANDMARKS}")
print(f"   Input shape: {INPUT_SHAPE}")
print(f"   Training on: ISLR 200 words ONLY")

In [ ]:
# Cell 3: Load ISLR 250-word Mapping
with open(ISLR_MAPPING_PATH, 'r') as f:
    islr_full_mapping = json.load(f)  # word -> label (0-249)

print(f"✅ ISLR mapping loaded: {len(islr_full_mapping)} words")
print(f"   First 10 words: {list(islr_full_mapping.keys())[:10]}")
print(f"   Label range: {min(islr_full_mapping.values())} - {max(islr_full_mapping.values())}")

# Create reverse mapping (label -> word)
islr_label_to_word = {v: k for k, v in islr_full_mapping.items()}

print(f"\n🎯 Will train on 200 words present in TFRecord files")

In [ ]:
# Cell 4: TFRecord Parsing Functions
def parse_tfrecord_example(example_proto):
    """Parse a single TFRecord example from ISLR dataset"""
    feature_description = {
        'landmarks': tf.io.FixedLenFeature([], tf.string),
        'label': tf.io.FixedLenFeature([], tf.int64),
    }
    
    parsed = tf.io.parse_single_example(example_proto, feature_description)
    
    # Decode landmarks: variable frames x 543 landmarks x 3 coords
    landmarks = tf.io.decode_raw(parsed['landmarks'], tf.float32)
    landmarks = tf.reshape(landmarks, [-1, TOTAL_LANDMARKS, 3])
    
    label = parsed['label']
    
    return landmarks, label

def resample_to_target_frames(landmarks, target_frames=TARGET_FRAMES):
    """Resample variable-length sequence to fixed 64 frames"""
    current_frames = tf.shape(landmarks)[0]
    
    # If already correct length
    if current_frames == target_frames:
        return tf.reshape(landmarks, [target_frames, -1])
    
    # If longer: sample evenly spaced frames
    if current_frames >= target_frames:
        indices = tf.linspace(0.0, tf.cast(current_frames-1, tf.float32), target_frames)
        indices = tf.cast(indices, tf.int32)
        landmarks = tf.gather(landmarks, indices)
    else:
        # If shorter: pad with last frame
        padding = target_frames - current_frames
        last_frame = landmarks[-1:]
        padding_frames = tf.tile(last_frame, [padding, 1, 1])
        landmarks = tf.concat([landmarks, padding_frames], axis=0)
    
    # Flatten: (64, 543, 3) -> (64, 1629)
    return tf.reshape(landmarks, [target_frames, -1])

print("✅ TFRecord parsing functions ready")

In [ ]:
# Cell 5: Load ISLR Training Data (10-15 minutes)
def load_islr_tfrecord(tfrecord_path, dataset_name="dataset"):
    """Load and process ISLR TFRecord file"""
    print(f"\n📦 Loading {dataset_name} from {tfrecord_path}...")
    
    dataset = tf.data.TFRecordDataset(tfrecord_path)
    
    X_data = []
    y_data = []
    label_counts = {}
    skipped = 0
    
    for record in tqdm(dataset, desc=f"Processing {dataset_name}"):
        try:
            landmarks, label = parse_tfrecord_example(record)
            
            # Convert to numpy
            landmarks_np = landmarks.numpy()
            label_np = int(label.numpy())
            
            # Resample to 64 frames
            processed = resample_to_target_frames(tf.constant(landmarks_np)).numpy()
            
            X_data.append(processed)
            y_data.append(label_np)
            
            # Track label distribution
            label_counts[label_np] = label_counts.get(label_np, 0) + 1
            
        except Exception as e:
            skipped += 1
            continue
    
    X_data = np.array(X_data, dtype=np.float32)
    y_data = np.array(y_data, dtype=np.int32)
    
    print(f"   ✅ Loaded: {len(X_data):,} samples")
    print(f"   ⚠️  Skipped: {skipped} samples")
    print(f"   📊 Unique labels: {len(label_counts)}")
    print(f"   📊 Shape: {X_data.shape}")
    
    return X_data, y_data, label_counts

# Load training data
print("🚀 Loading ISLR training data (this will take 10-15 minutes)...")
X_train, y_train, train_label_counts = load_islr_tfrecord(ISLR_TRAIN_PATH, "train")

# Load validation data
print("\n🚀 Loading ISLR validation data...")
X_val, y_val, val_label_counts = load_islr_tfrecord(ISLR_VAL_PATH, "val")

# Combine label counts
all_labels = set(train_label_counts.keys()).union(set(val_label_counts.keys()))
num_classes = len(all_labels)

print(f"\n✅ ISLR DATASET LOADED:")
print(f"   Train: {len(X_train):,} samples")
print(f"   Val: {len(X_val):,} samples")
print(f"   Total: {len(X_train) + len(X_val):,} samples")
print(f"   Classes: {num_classes} words")

# Show samples per class
all_samples = np.concatenate([y_train, y_val])
unique_labels, counts = np.unique(all_samples, return_counts=True)
print(f"\n📊 Samples per class:")
print(f"   Min: {counts.min()}")
print(f"   Max: {counts.max()}")
print(f"   Mean: {counts.mean():.1f}")
print(f"   Median: {np.median(counts):.0f}")

In [ ]:
# Cell 6: Create Final Label Mapping (Remap to 0-199)
# ISLR labels might not be continuous (0-199), so we remap them
unique_labels_sorted = sorted(np.unique(np.concatenate([y_train, y_val])))

# Create mapping: old_label -> new_label (0 to num_classes-1)
old_to_new_label = {old: new for new, old in enumerate(unique_labels_sorted)}
new_to_old_label = {new: old for old, new in old_to_new_label.items()}

# Remap labels
y_train_remapped = np.array([old_to_new_label[label] for label in y_train], dtype=np.int32)
y_val_remapped = np.array([old_to_new_label[label] for label in y_val], dtype=np.int32)

# Create final word mappings
final_label_to_word = {}
final_word_to_label = {}

for new_label, old_label in new_to_old_label.items():
    word = islr_label_to_word.get(old_label, f"unknown_{old_label}")
    final_label_to_word[str(new_label)] = word
    final_word_to_label[word] = new_label

print(f"✅ Labels remapped to continuous range [0-{num_classes-1}]")
print(f"   Train labels: {y_train_remapped.min()} - {y_train_remapped.max()}")
print(f"   Val labels: {y_val_remapped.min()} - {y_val_remapped.max()}")
print(f"\n📚 Vocabulary (first 20 words):")
for i in range(min(20, num_classes)):
    print(f"   {i}: {final_label_to_word[str(i)]}")

In [ ]:
# Cell 7: Data Quality Check
print("🔍 Data quality check:")
print(f"\n📊 Training data:")
print(f"   Shape: {X_train.shape}")
print(f"   Range: [{X_train.min():.3f}, {X_train.max():.3f}]")
print(f"   Mean: {X_train.mean():.3f}")
print(f"   Std: {X_train.std():.3f}")
print(f"   NaN values: {np.isnan(X_train).sum()}")
print(f"   Inf values: {np.isinf(X_train).sum()}")

print(f"\n📊 Validation data:")
print(f"   Shape: {X_val.shape}")
print(f"   Range: [{X_val.min():.3f}, {X_val.max():.3f}]")
print(f"   Mean: {X_val.mean():.3f}")
print(f"   Std: {X_val.std():.3f}")
print(f"   NaN values: {np.isnan(X_val).sum()}")
print(f"   Inf values: {np.isinf(X_val).sum()}")

print(f"\n✅ Data is clean and ready for training!")

In [ ]:
# Cell 8: Build Enhanced LSTM Model
model = Sequential([
    # First LSTM layer
    LSTM(128, return_sequences=True, activation='tanh', input_shape=INPUT_SHAPE),
    BatchNormalization(),
    Dropout(0.3),
    
    # Second LSTM layer
    LSTM(256, return_sequences=True, activation='tanh'),
    BatchNormalization(),
    Dropout(0.3),
    
    # Third LSTM layer
    LSTM(128, return_sequences=False, activation='tanh'),
    BatchNormalization(),
    Dropout(0.4),
    
    # Dense layers
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.4),
    
    Dense(128, activation='relu'),
    Dropout(0.3),
    
    # Output layer
    Dense(num_classes, activation='softmax')
])

# Compile model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print(f"✅ LSTM Model built:")
print(f"   Input: {INPUT_SHAPE}")
print(f"   Output: {num_classes} classes")
print(f"   Parameters: {model.count_params():,}")
print(f"   Model size: ~{model.count_params() * 4 / 1024 / 1024:.1f} MB")

model.summary()

In [ ]:
# Cell 9: Train Model (45-90 minutes)
# Setup callbacks
checkpoint = ModelCheckpoint(
    '/kaggle/working/lstm_islr200_best.h5',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.3,
    patience=8,
    min_lr=1e-7,
    verbose=1
)

early_stop = EarlyStopping(
    monitor='val_accuracy',
    patience=20,
    restore_best_weights=True,
    verbose=1,
    mode='max'
)

print("🚀 Starting training...")
print(f"   Training samples: {len(X_train):,}")
print(f"   Validation samples: {len(X_val):,}")
print(f"   Batch size: 64")
print(f"   Max epochs: 150")
print(f"   Estimated time: 45-90 minutes")
print(f"\n⏳ Please wait...\n")

history = model.fit(
    X_train, y_train_remapped,
    validation_data=(X_val, y_val_remapped),
    epochs=150,
    batch_size=64,
    callbacks=[checkpoint, reduce_lr, early_stop],
    verbose=1
)

print("\n✅ Training complete!")
print(f"   Total epochs: {len(history.history['loss'])}")
print(f"   Best val accuracy: {max(history.history['val_accuracy'])*100:.2f}%")
print(f"   Final train accuracy: {history.history['accuracy'][-1]*100:.2f}%")

In [ ]:
# Cell 10: Evaluate Model
# Load best model
best_model = tf.keras.models.load_model('/kaggle/working/lstm_islr200_best.h5')

print("🔍 Evaluating model performance...\n")

# Evaluate on validation set
val_loss, val_acc = best_model.evaluate(X_val, y_val_remapped, verbose=0)

# Get predictions
y_pred = best_model.predict(X_val, verbose=0)
y_pred_classes = np.argmax(y_pred, axis=1)

# Calculate top-k accuracies
top3_acc = top_k_accuracy_score(y_val_remapped, y_pred, k=3)
top5_acc = top_k_accuracy_score(y_val_remapped, y_pred, k=5)
top10_acc = top_k_accuracy_score(y_val_remapped, y_pred, k=10)

print(f"🎯 FINAL RESULTS:")
print(f"   📊 Validation Loss: {val_loss:.4f}")
print(f"   🎯 Top-1 Accuracy: {val_acc*100:.2f}%")
print(f"   🥉 Top-3 Accuracy: {top3_acc*100:.2f}%")
print(f"   🥇 Top-5 Accuracy: {top5_acc*100:.2f}%")
print(f"   🏆 Top-10 Accuracy: {top10_acc*100:.2f}%")

# Sample predictions
print(f"\n🔍 Sample predictions (first 20):")
for i in range(min(20, len(y_val_remapped))):
    true_word = final_label_to_word[str(y_val_remapped[i])]
    pred_word = final_label_to_word[str(y_pred_classes[i])]
    confidence = y_pred[i][y_pred_classes[i]] * 100
    status = "✅" if y_val_remapped[i] == y_pred_classes[i] else "❌"
    print(f"   {status} True: {true_word:15s} | Pred: {pred_word:15s} ({confidence:.1f}%)")

# Per-class accuracy
print(f"\n📈 Per-class statistics:")
class_accuracies = []
poor_classes = []

for label in range(num_classes):
    mask = y_val_remapped == label
    if mask.sum() > 0:
        acc = (y_pred_classes[mask] == label).mean()
        class_accuracies.append(acc)
        word = final_label_to_word[str(label)]
        
        if acc < 0.5:  # Poor performing classes
            poor_classes.append((word, acc, mask.sum()))

print(f"   Best class: {max(class_accuracies)*100:.1f}%")
print(f"   Worst class: {min(class_accuracies)*100:.1f}%")
print(f"   Average: {np.mean(class_accuracies)*100:.1f}%")

if poor_classes:
    print(f"\n⚠️  Classes with <50% accuracy:")
    for word, acc, count in sorted(poor_classes, key=lambda x: x[1])[:10]:
        print(f"   {word:15s}: {acc*100:.1f}% ({count} samples)")

In [ ]:
# Cell 11: Save Model and Deployment Package
print("💾 Saving model and creating deployment package...\n")

# Save model
best_model.save('/kaggle/working/lstm_islr200_final.h5')
best_model.save('/kaggle/working/lstm_islr200_savedmodel')

# Create deployment configuration
deployment_config = {
    'model_info': {
        'name': 'ISLR 200-Word LSTM Model',
        'version': '1.0',
        'framework': 'TensorFlow/Keras',
        'input_shape': list(INPUT_SHAPE),
        'target_frames': TARGET_FRAMES,
        'total_landmarks': TOTAL_LANDMARKS,
        'num_classes': num_classes,
        'training_samples': len(X_train),
        'validation_samples': len(X_val)
    },
    'performance': {
        'val_accuracy': float(val_acc),
        'val_loss': float(val_loss),
        'top3_accuracy': float(top3_acc),
        'top5_accuracy': float(top5_acc),
        'top10_accuracy': float(top10_acc),
        'expected_realworld_accuracy': f"{val_acc*0.8*100:.1f}-{val_acc*0.9*100:.1f}%"
    },
    'labels': {
        'word_to_label': final_word_to_label,
        'label_to_word': final_label_to_word,
        'num_classes': num_classes
    },
    'preprocessing': {
        'target_frames': TARGET_FRAMES,
        'landmarks_per_frame': TOTAL_LANDMARKS,
        'coordinates_per_landmark': 3,
        'input_flattened': True,
        'landmark_order': 'face(468) + left_hand(21) + pose(33) + right_hand(21)'
    }
}

# Save deployment config
with open('/kaggle/working/deployment_config.json', 'w') as f:
    json.dump(deployment_config, f, indent=2)

# Save training history
history_data = {
    'loss': [float(x) for x in history.history['loss']],
    'accuracy': [float(x) for x in history.history['accuracy']],
    'val_loss': [float(x) for x in history.history['val_loss']],
    'val_accuracy': [float(x) for x in history.history['val_accuracy']]
}

with open('/kaggle/working/training_history.json', 'w') as f:
    json.dump(history_data, f, indent=2)

# Create README
readme = f"""# ISLR 200-Word Sign Language Recognition Model

## Model Performance
- **Validation Accuracy:** {val_acc*100:.2f}%
- **Top-5 Accuracy:** {top5_acc*100:.2f}%
- **Training Samples:** {len(X_train):,}
- **Vocabulary:** {num_classes} words

## Model Specifications
- **Input:** {TARGET_FRAMES} frames × {TOTAL_LANDMARKS} landmarks × 3 coords = {INPUT_SHAPE}
- **Architecture:** 3-layer LSTM with BatchNormalization
- **Parameters:** {model.count_params():,}
- **Model Size:** ~{model.count_params() * 4 / 1024 / 1024:.1f} MB

## Usage
1. Load model: `model = tf.keras.models.load_model('lstm_islr200_final.h5')`
2. Load labels: `with open('deployment_config.json') as f: config = json.load(f)`
3. Preprocess: Extract MediaPipe landmarks, resample to 64 frames, flatten to (64, 1629)
4. Predict: `prediction = model.predict(landmarks)`

## Files
- `lstm_islr200_final.h5` - Keras model
- `lstm_islr200_savedmodel/` - TensorFlow SavedModel format
- `deployment_config.json` - Complete configuration and labels
- `training_history.json` - Training metrics
- `README.md` - This file
"""

with open('/kaggle/working/README.md', 'w') as f:
    f.write(readme)

print("✅ DEPLOYMENT PACKAGE CREATED:")
print("   📦 lstm_islr200_final.h5")
print("   📦 lstm_islr200_savedmodel/")
print("   📦 deployment_config.json")
print("   📦 training_history.json")
print("   📦 README.md")

print(f"\n🎯 MODEL SUMMARY:")
print(f"   🎪 Dataset: ISLR 200 words")
print(f"   📊 Training samples: {len(X_train):,}")
print(f"   🎯 Validation accuracy: {val_acc*100:.1f}%")
print(f"   🥇 Top-5 accuracy: {top5_acc*100:.1f}%")
print(f"   📚 Vocabulary: {num_classes} words")
print(f"   ⚡ Input: {TARGET_FRAMES} frames ({TARGET_FRAMES/30:.1f}s @ 30fps)")
print(f"   💾 Model size: ~{model.count_params() * 4 / 1024 / 1024:.1f} MB")

print(f"\n📥 DOWNLOAD ALL FILES FROM /kaggle/working/ FOR DEPLOYMENT")
print(f"\n🎉 TRAINING COMPLETE! Model ready for real-time inference.")